In [18]:
import pandas as pd
import numpy as np
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from shapely.geometry import box, mapping
import os
from glob import glob
from osgeo import gdal
import json

gdal.UseExceptions()

In [19]:
# for the S1 VV and VH mosaics:
#   - clip to AOI
#   - save to new files
# for the arctic DEM tiles:
#   - select grid shapefile tiles that intersect AOI and save tile numbers
#   - select DEM tiles that correspond to those tile numbers
#   - mosaic DEM tiles for each of aspect, slope, hillshade, elevation
#   - clip to AOI for each of aspect, slope, hillshade, elevation
#   - save to new files
# for the S2 MSI tiles:
#   - read in grid CSV and select gridcells that intersect AOI and save cell numbers
#   - select S2 MSI tiles that correspond to those cell numbers
#   - mosaic S2 MSI tiles for each of B02, B03, B04, B05, B06, B07, B8A, B11, B12
#   - clip to AOI for each of B02, B03, B04, B05, B06, B07, B8A, B11, B12
#   - save to new files

# Paths to files

In [20]:
# S1 paths
outdir = "/mnt/poseidon/remotesensing/arctic/data/rasters/s2_sr/tmp/out"
s1_tiles = sorted(glob("/mnt/poseidon/remotesensing/arctic/data/rasters/s1_grd_tiled/*.tif"))

In [21]:
# S2 MSI paths
grid_csv = "/mnt/poseidon/remotesensing/arctic/data/rasters/s2_sr/panarctic_grid/panarctic_0p25_gridcells.csv"
s2_base_pth = "/mnt/poseidon/remotesensing/arctic/data/rasters/s2_sr/composites"

In [22]:
# ArcticDEM paths
grid_shp = "/mnt/poseidon/remotesensing/arctic/data/vectors/supplementary/tundra_alaska_grid_latlon_aoi/tundra_alaska_grid_aoi.shp"
dem_aspect_tiles = glob("/mnt/poseidon/remotesensing/arctic/data/rasters/arctic_dem/*_dem_aspect.tif")
dem_slope_tiles = glob("/mnt/poseidon/remotesensing/arctic/data/rasters/arctic_dem/*_dem_slope.tif")
dem_hillshade_tiles = glob("/mnt/poseidon/remotesensing/arctic/data/rasters/arctic_dem/*_dem_hillshade.tif")
dem_elevation_tiles = glob("/mnt/poseidon/remotesensing/arctic/data/rasters/arctic_dem/*_dem.tif")

# Basic Parameters

In [23]:
# params
bbox = box(-150, 67, -148, 69)
xres = 0.000179663056824 
yres = 0.000179663056824
epsg = 'EPSG:4326'

In [24]:
bbox = box(-150, 67, -148, 69)
aoi_geojson = {
    "type": "FeatureCollection",
    "name": "aoi",
    "crs": {"type": "name", "properties": {"name": "EPSG:4326"}},
    "features": [{"type": "Feature", "properties": {}, "geometry": mapping(bbox)}],
}

# Write to GDAL's in-memory filesystem
aoi_path = "/vsimem/aoi.geojson"
_ = gdal.FileFromMemBuffer(aoi_path, json.dumps(aoi_geojson))

# Prep S1

In [25]:
# select gridcells from shp that intersect AOI
grid = gpd.read_file(grid_shp)
grid = grid.to_crs(epsg)
grid_aoi = grid[grid.intersects(bbox)]
grid_aoi_ids = grid_aoi["FID"].tolist()

# select tifs that correspond to those gridcells
def select_tiles(tiles, ids, band):
    selected = []
    for tile in tiles:
        for id in ids:
            if f"GRIDCELL_{id}_{band}" in tile:
                selected.append(tile)
    return selected
s1_vv_aoi = select_tiles(s1_tiles, grid_aoi_ids, "VV")
s1_vh_aoi = select_tiles(s1_tiles, grid_aoi_ids, "VH")

In [26]:
# gdal progress callback
def gdal_progress_cb(complete, message, userdata):
    label = userdata or ""
    gdal.TermProgress_nocb(complete, label)
    return 1  # non-zero to continue, 0 would abort

# gdal warp creation options
wo = {"NUM_THREADS": "ALL_CPUS", "INIT_DEST": "NO_DATA"}
co = [
    "TILED=YES",
    "COMPRESS=ZSTD",
    "ZSTD_LEVEL=15",
    "PREDICTOR=3",
    "BIGTIFF=YES",
    "BLOCKXSIZE=512",
    "BLOCKYSIZE=512",
]

# gdal warp kwargs
kw = dict(
    cutlineDSName=aoi_path,
    cutlineSRS="EPSG:4326",
    cropToCutline=True,
    dstSRS=epsg,
    outputBounds=bbox.bounds,
    outputBoundsSRS="EPSG:4326",
    xRes=xres,
    yRes=yres,
    targetAlignedPixels=True,
    outputType=gdal.GDT_Float32,
    resampleAlg="bilinear",
    dstNodata=-9999,
    creationOptions=co,
    warpOptions=wo,
)

In [13]:
# create temporary mosaic of selected tiles
vv_vrt = "/vsimem/s1_vv_mosaic.vrt"
vh_vrt = "/vsimem/s1_vh_mosaic.vrt"
ds = gdal.BuildVRT(vv_vrt, s1_vv_aoi,
                   callback=gdal_progress_cb, callback_data="[BuildVRT VV] ")
assert ds is not None, "BuildVRT VV returned None"
ds = None
assert gdal.VSIStatL(vv_vrt) is not None, "VV VRT not written"

ds = gdal.BuildVRT(vh_vrt, s1_vh_aoi,
                   callback=gdal_progress_cb, callback_data="[BuildVRT VH] ")
assert ds is not None, "BuildVRT VH returned None"
ds = None
assert gdal.VSIStatL(vh_vrt) is not None, "VH VRT not written"

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50..

.60...70...80...90...100 - done.


In [ ]:
# clip and save to new files
vv_tif_aoi = os.path.join(outdir, "S1_VV_AOI.tif")
vh_tif_aoi = os.path.join(outdir, "S1_VH_AOI.tif")

gdal.Warp(vv_tif_aoi, vv_vrt, callback=gdal_progress_cb, callback_data="[Warp VV] ", **kw)
gdal.Warp(vh_tif_aoi, vh_vrt, callback=gdal_progress_cb, callback_data="[Warp VH] ", **kw)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f976c5099b0> >

In [ ]:
# clean up
_ = gdal.Unlink(vv_vrt)
_ = gdal.Unlink(vh_vrt)

# Prep S2


In [27]:
# choose S2 gridcells that intersect AOI
# CSV layout:
# tile_id_rc,tile_id_geo,xmin,ymin,xmax,ymax,row,col
# Q025_R000_C0000,Q025_W180_00_N89_75,-180.000000,89.750000,-179.750000,90.000000,0,0
grid = pd.read_csv(grid_csv)
grid["geometry"] = grid.apply(lambda row: box(row.xmin, row.ymin, row.xmax, row.ymax), axis=1)
grid = gpd.GeoDataFrame(grid, geometry="geometry", crs="EPSG:4326")
grid_aoi = grid[grid.intersects(bbox)]
grid_aoi_ids = grid_aoi["tile_id_rc"].tolist()

In [29]:
# gdal warp creation options
wo = {"NUM_THREADS": "ALL_CPUS", "INIT_DEST": "NO_DATA"}
co = [
    "TILED=YES",
    "COMPRESS=ZSTD",
    "ZSTD_LEVEL=15",
    "PREDICTOR=2",
    "BIGTIFF=YES",
    "BLOCKXSIZE=512",
    "BLOCKYSIZE=512",
]

# gdal warp kwargs
kw = dict(
    cutlineDSName=aoi_path,
    cutlineSRS="EPSG:4326",
    cropToCutline=True,
    dstSRS=epsg,
    outputBounds=bbox.bounds,
    outputBoundsSRS="EPSG:4326",
    xRes=xres,
    yRes=yres,
    targetAlignedPixels=True,
    outputType=gdal.GDT_UInt16,
    resampleAlg="nearest",
    dstNodata=65535,
    creationOptions=co,
    warpOptions=wo,
)

In [30]:
# loop through S2 bands
bands = ["B02", "B03", "B04", "B05", "B06", "B07", "B8A", "B11", "B12"]
for band in bands:

    # select tifs that correspond to those gridcells
    def select_tiles(tiles, ids, band):
        selected = []
        for tile in tiles:
            for id in ids:
                if f"cell_{id}_{band}" in tile:
                    selected.append(tile)
        return selected
    s2_tiles = sorted(glob(f"{s2_base_pth}/*/cell_*_{band}_median_clip.tif"))
    s2_aoi = select_tiles(s2_tiles, grid_aoi_ids, band)

    # create temporary mosaic of selected tiles
    s2_vrt = f"/vsimem/s2_{band}_mosaic.vrt"
    ds = gdal.BuildVRT(s2_vrt, s2_aoi,
                       callback=gdal_progress_cb, callback_data=f"[BuildVRT {band}] ")
    assert ds is not None, f"BuildVRT {band} returned None"
    ds = None
    assert gdal.VSIStatL(s2_vrt) is not None, f"{band} VRT not written"

    # clip and save to new files
    s2_tif_aoi = os.path.join(outdir, f"S2_{band}_AOI.tif")
    gdal.Warp(s2_tif_aoi, s2_vrt, callback=gdal_progress_cb, callback_data=f"[Warp {band}] ", **kw)
    
    # clean up
    _ = gdal.Unlink(s2_vrt)

..40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80

# Prep ArcticDEM